In [ ]:
from ipywidgets import widgets
import numpy as np
from functools import partial
from zipfile import ZipFile
from h5py import File as h5py_File
from os import remove

In [ ]:
width_png=547
height_png=400

In [ ]:
image_height=250
image_width=int(width_png*image_height/height_png)
tests_results_width=300

In [ ]:
s_data_file_name='data_for_ex3'

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    system('wget https://github.com/patrickhaddadteaching/TRNG_ex3/raw/main/data/data_for_ex3.zip')    

In [ ]:
IN_COLAB

In [ ]:
path_to_zip_file='%s.zip'%(s_data_file_name)
zip_ref=ZipFile(path_to_zip_file, 'r')
zip_ref.extractall('.')

In [ ]:
hdf5_file = '%s.h5'%s_data_file_name
data_file = h5py_File(hdf5_file,'r')
v_ratio_period_sample_for_interact=np.array(data_file['parameters/v_ratio_period_sample'][0,:],dtype=np.uint32)
v_jitter_for_interact=data_file['parameters/v_jitter'][0,:]
v_raw_rng_image=data_file['images/rng_image'][0,:]
m_proba1=data_file['raw_proba/m_proba1'][:]
data_file.close()
remove('%s'%hdf5_file)
nb_osc=m_proba1.shape[0]

In [ ]:
def compute_proba_xor(m_value,i_jitter,i_per_sample):
    pxor=0.5+((-2)**(m_value-1))*(m_proba1[:m_value,i_jitter,i_per_sample]-0.5).prod()
    return pxor

In [ ]:
tests_results_value='<p><b>Shannon entropy VALUE: H<sub>1</sub>=</b></p>'
tests_results_value='%s<p><b>Min entropy VALUE: H<sub>&#8734</sub>=</b></p>'%(tests_results_value)

v_std_dev_value_widgets_option=[]
for i in range(v_jitter_for_interact.shape[0]):
    v_std_dev_value_widgets_option.append('%.2e'%(v_jitter_for_interact[i]))
std_dev_value_widgets=widgets.SelectionSlider(description=r'$\frac{\sigma_{tot}}{T_{1}}$',options=v_std_dev_value_widgets_option,layout=widgets.Layout(width='%dpx'%(image_width)))

sample_period_value_widgets=widgets.SelectionSlider(description=r'$D$',options=list(v_ratio_period_sample_for_interact),layout=widgets.Layout(width='%dpx'%(image_width)))
m_value_widgets=widgets.SelectionSlider(description=r'$m$',options=list(np.arange(nb_osc,dtype=np.uint8)+1),layout=widgets.Layout(width='%dpx'%(image_width)))
rng_image=widgets.Image(value=v_raw_rng_image.tobytes(), format='png',layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%image_width))
tests_results=widgets.HTML(value=tests_results_value,placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%(image_height/6),width='auto'))

space_for_tests_results_left=widgets.HTML(value='',placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%(image_height/3),width='%dpx'%(image_height/8) ))
space_for_tests_results_top=widgets.HTML(value='',placeholder='',description='',disabled=True,layout=tests_results.layout)

def change_on_param(obj):
    if obj['new']:
        px=compute_proba_xor(int(m_value_widgets.value),v_std_dev_value_widgets_option.index(std_dev_value_widgets.value),np.where(sample_period_value_widgets.value==v_ratio_period_sample_for_interact)[0][0])
        if (px*(1-px))==0:
            hs=0
            hm=0
        else:
            hs=((px)*np.log2(px)+(1-px)*np.log2(1-px))*-1
            hm=-1*np.log2(max([px,1-px]))
            
        tests_results_value='<p><b>Shannon entropy VALUE: H<sub>1</sub>=%.3f </b></p>'%(hs)
        tests_results_value='%s<p><b>Min entropy VALUE: H<sub>&#8734</sub>=%.3f</b></p>'%(tests_results_value,hm)
        tests_results.value=tests_results_value

        
m_value_widgets.observe(partial(change_on_param),'value')
sample_period_value_widgets.observe(partial(change_on_param),'value')
std_dev_value_widgets.observe(partial(change_on_param),'value')

v_box_top=widgets.HBox([rng_image,widgets.VBox([space_for_tests_results_top,widgets.HBox([space_for_tests_results_left,tests_results]),m_value_widgets,std_dev_value_widgets,sample_period_value_widgets])])
display(v_box_top)